In [2]:
import math
import matplotlib.pyplot as plt
import ROOT
import copy


from ROOT import TCanvas, TFile, TProfile, TNtuple, TH1F, TH2F, TF1, TLegend, gStyle, TF2, TCutG
from ROOT import gROOT, gBenchmark, gRandom, gSystem

import numpy as np
from uits3_krakow22.src.Event import Event
from uits3_krakow22.src.Cluster import Cluster
from uits3_krakow22.src.Track import Track
from uits3_krakow22.src.Vertex import Vertex
from uits3_krakow22.src.Utils import *

from ipywidgets import IntProgress
import matplotlib.pyplot as plt
from IPython.display import display,HTML,clear_output
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>table {float:left;}</style>"))

Welcome to JupyROOT 6.24/06


# Load Data

In [4]:
#path = "/home/cehrich/Software/uits3_krakow22/uits3_krakow22/output/yze6run456065424_221112065722.pkl" #200 MeV
#path = "/home/cehrich/Software/uits3_krakow22/uits3_krakow22/output/yze6run457063715_221113063928.pkl" #200 MeV
#path = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run456114514_221112120005.pkl"
#path = "/home/cehrich/Software/uits3_krakow22/uits3_krakow22/output/yzrun457063715_221113063928.pkl" #full file, up up
#path = "/home/cehrich/Software/uits3_krakow22/run457063715_221113063928_clusters.pkl" # full file up down
#path = "/home/cehrich/Software/uits3_krakow22/uits3_krakow22/output/yzAlign_run456195948_221112200957.pkl"
#path = "/home/cehrich/Software/uits3_krakow22/run456195948_221112200957.pkl"


path = "/home/cehrich/Software/uits3_krakow22/pickleNeedsFlip/run456195948_221112200957.pkl" #200MeV 100e
nEvents = -1
selectedEvents = []
if nEvents > 0 :
    f = IntProgress(min=0, max=nEvents, description="Processing:")
    display(f)

for i,event in enumerate(readEvents(path, nEvents=nEvents)):
    #print("test")
    if nEvents > 0 and i>0 and i%1e4 == 0: f.value += 1e4
    eventStatus = True
    event.clusters = [cluster for cluster in event.clusters if 7 < cluster.size < 19]
    for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]:
        clusters = event.selectDetector(alpide)
        if len(clusters) != 1:
            eventStatus = False
            break
    if eventStatus : selectedEvents.append(event)
        
if 'f' in locals(): 
    f.bar_style = "success"
    print("Found events:",len(selectedEvents),", Fraction:",len(selectedEvents)*100/nEvents,"%")
path_selected = "/home/cehrich/Software/uits3_krakow22/selectedEvents.pkl"
save2pickle(selectedEvents,path_selected)

Going to process all events.


# Flip

In [5]:
selectedEventsFlipped = []
for event in selectedEvents:
    for cluster in event.clusters:
        cluster.flipYaxis()
    selectedEventsFlipped.append(event)

# Add Tracks

In [6]:
len(selectedEvents)

#path_selected = "/home/cehrich/Software/uits3_krakow22/selectedEvents.pkl" #this was defined already above

for event in selectedEventsFlipped:
    TrackLeft = Track()
    TrackLeft.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_3","ALPIDE_4"]])
    TrackRight = Track()
    TrackRight.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
    vertex = Vertex()
    vertex.fromTracks([TrackLeft,TrackRight])
    event.addTrack(TrackLeft)
    event.addTrack(TrackRight)
    event.vertex = vertex


    
    #events.append(event)
#selectedEvents)


In [75]:
tempEvents=copy.deepcopy(selectedEventsFlipped)
print("Number of events: ", len(tempEvents))


Number of events:  55865


In [76]:
%jsroot on

hVertexXY = ROOT.TH2F("VertexXY","VertexXY",200,-7,7,200,-7,7)
hVertexXY.GetXaxis().SetTitle("Vertex X position [mm]")
hVertexXY.GetYaxis().SetTitle("Vertex Y position [mm]")
hVertexXZ = ROOT.TH2F("VertexXZ","VertexXZ",120,-2,2,120,-21,-17)
hVertexXZ.GetXaxis().SetTitle("Vertex X position [mm]")
hVertexXZ.GetYaxis().SetTitle("Vertex Z position [mm]")
hVertexYZ = ROOT.TH2F("VertexYZ","VertexYZ",200,-7,7,200,-26,-12)
hVertexYZ.GetXaxis().SetTitle("Vertex Y position [mm]")
hVertexYZ.GetYaxis().SetTitle("Vertex Z position [mm]")


print("Number of events: ", len(tempEvents))

#eventsFlipped= []
alpides=["ALPIDE_0","ALPIDE_1","ALPIDE_2", "ALPIDE_3","ALPIDE_4"]

displacement={ #angular displacements
    'ALPIDE_0': 0.34375, 
    'ALPIDE_1': 0.40625, 
    'ALPIDE_2': 0.375, 
    'ALPIDE_3': 0.0, 
    'ALPIDE_4': 0.251953125
} #[0.35, -0.0625, 0], [0.525, -0.0625, 0], [0.6, -0.125, 0], [0.35, 0, 0], [0.7, -0.0625, 0]

# with [0.06201674114823931, 0.48295208736402606]
best_displacement_no_multiple_scat ={
    'ALPIDE_0': [0.5, -0.0546875, 0], 
    'ALPIDE_1': [0.65, -0.0546875, 0], 
    'ALPIDE_2': [0.75, -0.06875, 0], 
    'ALPIDE_3': [0.2, -0.125, 0], 
    'ALPIDE_4': [0.5546875, -0.05, 0]
}

berkin_ml_displacement={
    'ALPIDE_0': [0.20006977021694183, 0.8966014385223389], 
    'ALPIDE_1': [-0.7433865666389465, 0.8977992534637451], 
    'ALPIDE_2': [0.8155627250671387, 0.9625105261802673], 
    'ALPIDE_3': [-0.10266361385583878, -0.31308597326278687], 
    'ALPIDE_4': [-0.4219255745410919, -1.322750449180603]
}

custom_displacement={
     'ALPIDE_0': [0, 0, 0], 
     'ALPIDE_1': [1.1, 0, 0], 
     'ALPIDE_2': [0.2, 0, 0], 
     'ALPIDE_3': [0, 0, 0], 
     'ALPIDE_4': [0, 0, 0]
}

fourRound15DAlignment={
    'ALPIDE_0': [-0.5, -0.5, 0, 1.09375], 
    'ALPIDE_1': [-0.8125, -0.8125, 0, 1.0625], 
    'ALPIDE_2': [-0.875, -0.8125, 0, 1.125], 
    'ALPIDE_3': [-0.25, -0.8125, 0, 1.0], 
    'ALPIDE_4': [-0.875, -0.75, 0, 1.09375]                                             
} #. Got median DCA  0.05901272 . Got median track RMS  0.48295208  and mean DCA:  0.089  and DCA sigma:  0.130

displacement_angles_refined_alignment={ #angular displacements 
    'ALPIDE_0': 0.34375, 
    'ALPIDE_1': 0.40625, 
    'ALPIDE_2': 0.375, 
    'ALPIDE_3': 0.0, 
    'ALPIDE_4': 0.251953125
}
fourRound15DAlignmentAngles={ #angular displacements
    'ALPIDE_0': 1.09375, 
    'ALPIDE_1': 1.0625, 
    'ALPIDE_2': 1.125, 
    'ALPIDE_3': 1.0, 
    'ALPIDE_4': 1.09375
}

alignment16082023={ #displacements
    'ALPIDE_0': [-0.30625, 0.0875, 0], 
    'ALPIDE_1': [-0.2625, 0.13125, 0],
    'ALPIDE_2': [0.35, 0.0875, 0], 
    'ALPIDE_3': [-0.35, 0.13125, 0],
    'ALPIDE_4': [-0.30625, 0.1421875, 0]
}
alignment16082023Angles={ #angular displacements
    'ALPIDE_0': -0.2953125, 
    'ALPIDE_1': 0.35, 
    'ALPIDE_2': -0.2953125, 
    'ALPIDE_3': -0.26249999999999996, 
    'ALPIDE_4': -0.30624999999999997
} # with [0.05733863420923402, 0.48295208736402606, 0.29881467789124705]


alignment23082023={ #displacements
    'ALPIDE_0': [-0.2994140625, -0.24375, 0], 
    'ALPIDE_1': [-0.225, -0.2625, 0],
    'ALPIDE_2': [0.3, -0.22558593749999997, 0], 
    'ALPIDE_3': [-0.2994140625, -0.24375, 0],
    'ALPIDE_4': [-0.2994140625, -0.225, 0]
}
alignment23082023Angles={ #angular displacements
    'ALPIDE_0': -0.3, 
    'ALPIDE_1': 0.3, 
    'ALPIDE_2': -0.16875, 
    'ALPIDE_3': -0.15, 
    'ALPIDE_4': -0.178125
} # with [0.07227255735846638, 0.057678879571534974, 0.48285208975322036, 0.31369860223507656]

alignment01092023={
     'ALPIDE_0': [-0.28125, -0.225, 0.15], 
     'ALPIDE_1': [-0.225, -0.24375, 0.15], 
     'ALPIDE_2': [0.3, -0.24375, 0.075], 
     'ALPIDE_3': [-0.3, -0.2625, 0.075], 
     'ALPIDE_4': [-0.3, -0.24375, 0.15]
}

localx_alignment1={
    'ALPIDE_0': [0, 0.3, 0], 
    'ALPIDE_1': [1.1, 0.23437499999999997, 0], 
    'ALPIDE_2': [0.2, 0.22499999999999998, 0], 
    'ALPIDE_3': [0, 0.26249999999999996, 0], 
    'ALPIDE_4': [0, 0.22499999999999998, 0]
}
localx_alignment= {
    'ALPIDE_0': [0, 0.0, 0],
    'ALPIDE_1': [1.1, 0.0, 0],
    'ALPIDE_2': [0.2, 0.0, 0],
    'ALPIDE_3': [0, 0.0, 0],
    'ALPIDE_4': [0, 0, 0]
} #Got median DCA  0.05568996 . Got median track RMS  0.48285208  and mean DCA:  0.072  and DCA sigma:  0.107


dcaVals = []
dcaValsX = [] # y and z
dcaValsY = [] 
dcaValsZ = [] 
trackRMSVals = []
evenMoreTempEvents = []


for index, event in enumerate(tempEvents):
    #event.selectDetector(alpide)[0].twistTopDown(displacement.get(alpide)) for alpide in alpides
    #event.selectDetector(alpide)[0].alignLocal(best_displacement_no_multiple_scat.get(alpide)) for alpide in alpides
    
    #TrackLeft = Track()
    #TrackLeft.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_3","ALPIDE_4"]])
    #TrackRight = Track()
    #TrackRight.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])

    
    #TrackLeft = Track()
    #TrackLeft.fromClusters([event.selectDetector(alpide)[0].twistTopDown(alignment23082023Angles.get(alpide)).alignLocal(alignment16082023.get(alpide)) for alpide in ["ALPIDE_3","ALPIDE_4"]])
    #TrackRight = Track()
    #TrackRight.fromClusters([event.selectDetector(alpide)[0].twistTopDown(alignment23082023Angles.get(alpide)).alignLocal(alignment16082023.get(alpide)) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])

    TrackLeft = Track()
    TrackLeft.fromClusters([event.selectDetector(alpide)[0].alignLocal(localy_alignment.get(alpide)) for alpide in ["ALPIDE_3","ALPIDE_4"]])
    #TrackLeft.fromClusters([event.selectDetector(alpide)[0].alignLocal(berkin_ml_displacement.get(alpide)) for alpide in ["ALPIDE_3","ALPIDE_4"]])

    TrackRight = Track()
    TrackRight.fromClusters([event.selectDetector(alpide)[0].alignLocal(localy_alignment.get(alpide)) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
    #TrackRight.fromClusters([event.selectDetector(alpide)[0].alignLocal(berkin_ml_displacement.get(alpide)) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
    #evenMoreTempEvents.append(event)
    
    #    
    #TrackLeft = Track()
    #TrackLeft.fromClusters([event.selectDetector(alpide)[0].twistTopDown(displacement.get(alpide)) for alpide in ["ALPIDE_3","ALPIDE_4"]])
    #TrackRight = Track()
    #TrackRight.fromClusters([event.selectDetector(alpide)[0].twistTopDown(displacement.get(alpide)) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
    #
    
    event.vertex = Vertex()
    event.vertex.fromTracks([TrackLeft,TrackRight])
    
    if (index%1000 ==0): print(str(index/len(tempEvents)*100)[:4], "%", end="\r")
    #event.addTrack(TrackLeft)
    #event.addTrack(TrackRight)
    #event.vertex = vertex
    #tempEvents.append(event)
    dcaVals.append(event.vertex.dca)
    dcaValsX.append(event.vertex.dcaVec[0])
    dcaValsY.append(event.vertex.dcaVec[1])
    dcaValsZ.append(event.vertex.dcaVec[2])

    for track in event.tracks:
        if track.nClusters == 3:
            trackRMS = track.rms
            trackRMSVals.append(trackRMS)
    
    hVertexXY.Fill(event.vertex.point[0],event.vertex.point[1]) #maybe apply some selection to the tracks, ask berkin
    hVertexXZ.Fill(event.vertex.point[0],event.vertex.point[2])
    hVertexYZ.Fill(event.vertex.point[1],event.vertex.point[2])

#def eventLoop(eventToLoup):
#    for index, event in enumerate(eventToLoup):
#    
#        TrackLeft = Track()
#        TrackLeft.fromClusters([event.selectDetector(alpide)[0].twistTopDown(displacement.get(alpide)).alignLocal(best_displacement_no_multiple_scat.get(alpide)) for alpide in ["ALPIDE_3","ALPIDE_4"]])
#        TrackRight = Track()
#        TrackRight.fromClusters([event.selectDetector(alpide)[0].twistTopDown(displacement.get(alpide)).alignLocal(best_displacement_no_multiple_scat.get(alpide)) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
#
#        event.vertex = Vertex()
#        event.vertex.fromTracks([TrackLeft,TrackRight])
#
#        if (index%1000 ==0): print(str(index/len(tempEvents)*100)[:4], "%", end="\r")
#        dcaVals.append(vertex.dca)
#        for track in event.tracks:
#            if track.nClusters == 3:
#                trackRMS = track.rms
#                trackRMSVals.append(trackRMS)
#
#        hVertexXY.Fill(event.vertex.point[0],event.vertex.point[1]) #maybe apply some selection to the tracks, ask berkin
#        hVertexXZ.Fill(event.vertex.point[0],event.vertex.point[2])
#        hVertexYZ.Fill(event.vertex.point[1],event.vertex.point[2])

        


    
medianDCA = np.median(dcaVals)
meanDCA = np.mean(dcaVals)
medianTrackRMS = np.median(trackRMSVals)
print("Evaluated displacement", custom_displacement,". \n Median DCA: ",str(medianDCA)[:8]," \n Mean DCA: ", str(meanDCA)[:8], " \n DCA sigma: ", str(np.std(dcaVals))[:8], " \n Median track RMS: ",str(medianTrackRMS)[:8])

canvas = ROOT.TCanvas("","",1300,500)
canvas.Divide(3)

canvas.cd(1)
ROOT.gPad.SetGrid(2)
hVertexXY.Draw("COLZ") 

canvas.cd(2)
ROOT.gPad.SetGrid(2)
hVertexXZ.Draw("COLZ")

canvas.cd(3)
ROOT.gPad.SetGrid(2)
hVertexYZ.Draw("COLZ")

canvas.Draw()

#canvas.SaveAs("many_events_canvas.png")
#canvas.Close()

Number of events:  55865
Evaluated displacement {'ALPIDE_0': [0, 0, 0], 'ALPIDE_1': [1.1, 0, 0], 'ALPIDE_2': [0.2, 0, 0], 'ALPIDE_3': [0, 0, 0], 'ALPIDE_4': [0, 0, 0]} . 
 Median DCA:  0.647140  
 Mean DCA:  5.213241  
 DCA sigma:  7.441431  
 Median track RMS:  0.483616


Warning in <TROOT::Append>: Replacing existing TH1: VertexXY (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: VertexXZ (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: VertexYZ (Potential memory leak).


## Very Selected Events

In [ ]:
%jsroot on

hVertexXY = ROOT.TH2F("VertexXY","VertexXY",200,-7,7,200,-7,7)
hVertexXY.GetXaxis().SetTitle("Vertex X position [mm]")
hVertexXY.GetYaxis().SetTitle("Vertex Y position [mm]")
hVertexXZ = ROOT.TH2F("VertexXZ","VertexXZ",120,-2,2,120,-21,-17)
hVertexXZ.GetXaxis().SetTitle("Vertex X position [mm]")
hVertexXZ.GetYaxis().SetTitle("Vertex Z position [mm]")
hVertexYZ = ROOT.TH2F("VertexYZ","VertexYZ",200,-7,7,200,-26,-12)
hVertexYZ.GetXaxis().SetTitle("Vertex Y position [mm]")
hVertexYZ.GetYaxis().SetTitle("Vertex Z position [mm]")
trackRMSVals=[]
verySelectedEvents=[]
dcaVals = []
dcaValsX = []
dcaValsY = []
dcaValsZ = []

path_clean = "/home/cehrich/Software/uits3_krakow22/cleanEvents_brute.pkl" #same events used in brute force alignment
for event in readEvents(path_clean, nEvents=-1):
    #200MeV 100e
    #if 85 < event.vertex.openingAngle*360/(2*math.pi) < 90 and abs(event.vertex.dca2origin - 18.75) < 2 and event.vertex.dca < 2 and all([5 < cluster.size < 25 for cluster in event.clusters]):#2sigma of 80MeV 100e target sizes
    #    if all([(-8 < cluster.localPos[0] < 8 and -3 < cluster.localPos[1] < 3) for cluster in event.clusters]):
    #
    #80MeV 100e
    #if 87 < event.vertex.openingAngle*360/(2*math.pi) < 92 and abs(event.vertex.dca2origin - 18.75) < 2 and event.vertex.dca < 2 and all([10 < cluster.size < 30 for cluster in event.clusters]):#2sigma of 80MeV 100e target sizes
    verySelectedEvents.append(event)
        
    TrackLeft = Track()
    TrackLeft.fromClusters([event.selectDetector(alpide)[0].alignLocal(alignment01092023.get(alpide)) for alpide in ["ALPIDE_3","ALPIDE_4"]])
    TrackRight = Track()
    TrackRight.fromClusters([event.selectDetector(alpide)[0].alignLocal(alignment01092023.get(alpide)) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])

    #TrackLeft = Track()
    #TrackLeft.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_3","ALPIDE_4"]])
    #TrackRight = Track()
    #TrackRight.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
    
    event.vertex = Vertex()
    event.vertex.fromTracks([TrackLeft,TrackRight])

    if (index%1000 ==0): print(str(index/len(tempEvents)*100)[:4], "%", end="\r")
    dcaVals.append(vertex.dca)
    dcaValsX.append(vertex.dcaVec[0])
    dcaValsY.append(vertex.dcaVec[1])
    dcaValsZ.append(vertex.dcaVec[2])
    for track in event.tracks:
        if track.nClusters == 3:
            trackRMS = track.rms
            trackRMSVals.append(trackRMS)

    
    hVertexXY.Fill(event.vertex.point[0],event.vertex.point[1]) #maybe apply some selection to the tracks, ask berkin
    hVertexXZ.Fill(event.vertex.point[0],event.vertex.point[2])
    hVertexYZ.Fill(event.vertex.point[1],event.vertex.point[2])
    
print("Number of very selected events: ", len(verySelectedEvents))



canvas = ROOT.TCanvas("","",1300,500)
canvas.Divide(3)

canvas.cd(1)
ROOT.gPad.SetGrid(2)
hVertexXY.Draw("COLZ") 

canvas.cd(2)
ROOT.gPad.SetGrid(2)
hVertexXZ.Draw("COLZ")

canvas.cd(3)
ROOT.gPad.SetGrid(2)
hVertexYZ.Draw("COLZ")

canvas.Draw()
evaluateEventSet(verySelectedEvents)
#canvas.SaveAs("selected_events_canvas.png")
#canvas.Close()

## My Custom Selected Events (no brutes here)

In [119]:
%jsroot on

hVertexXY = ROOT.TH2F("VertexXY","VertexXY",200,-7,7,200,-7,7)
hVertexXY.GetXaxis().SetTitle("Vertex X position [mm]")
hVertexXY.GetYaxis().SetTitle("Vertex Y position [mm]")
hVertexXZ = ROOT.TH2F("VertexXZ","VertexXZ",120,-2,2,120,-21,-17)
hVertexXZ.GetXaxis().SetTitle("Vertex X position [mm]")
hVertexXZ.GetYaxis().SetTitle("Vertex Z position [mm]")
hVertexYZ = ROOT.TH2F("VertexYZ","VertexYZ",200,-7,7,200,-26,-12)
hVertexYZ.GetXaxis().SetTitle("Vertex Y position [mm]")
hVertexYZ.GetYaxis().SetTitle("Vertex Z position [mm]")
trackRMSVals=[]
verySelectedEvents=[]
dcaVals = []
dcaValsX = []
dcaValsY = []
dcaValsZ = []

myEvents = []

null_displacement={
     'ALPIDE_0': [0, 0, 0], 
     'ALPIDE_1': [0, 0, 0], 
     'ALPIDE_2': [0, 0, 0], 
     'ALPIDE_3': [0, 0, 0], 
     'ALPIDE_4': [0, 0, 0]
}

updated_alignment = {
     'ALPIDE_0': [0, 0.290625, 0],
     'ALPIDE_1': [1.1, 0.0, 0],
     'ALPIDE_2': [0.2, 0.3, 0],
     'ALPIDE_3': [0, 0.3, 0],
     'ALPIDE_4': [0, -0.1875, 0]
}

#localxupdated_alignment={
#    'ALPIDE_0': [-0.755859375, -0.7265625, 0], 
#    'ALPIDE_1': [0.5, 0, 0], 
#    'ALPIDE_2': [-0.25634765625, 0, 0], 
#    'ALPIDE_3': [0.0, 1.0, 0], 
#    'ALPIDE_4': [0.048828125, 0.265625, 0]
#}
localxupdated_alignment={'ALPIDE_0': [0, -1.75, 0], 'ALPIDE_1': [1.1, 1.296875, 0], 'ALPIDE_2': [0.2, 1.0, 0], 'ALPIDE_3': [0, 1.75, 0], 'ALPIDE_4': [0, -1.25, 0]} 
#localxupdated_alignment= {'ALPIDE_0': [0, -1.74609375, 0], 'ALPIDE_1': [1.1, 1.296875, 0], 'ALPIDE_2': [0.2, 1.00390625, 0], 'ALPIDE_3': [0, 1.74609375, 0], 'ALPIDE_4': [0, -1.25390625, 0]}
xMin=-2
xMax=2

yMin=-5
yMax=5

zMin=-21
zMax=-17

for event in copy.deepcopy(selectedEventsFlipped): # events: 
    if 80 < event.vertex.openingAngle*360/(2*math.pi) < 95 and event.vertex.dca < 5 and all([7<cluster.size<20 for cluster in event.clusters]):
        if( xMin < event.vertex.point[0] < xMax and yMin < event.vertex.point[1] < yMax and zMin < event.vertex.point[2] < zMax):
            if(all([(-10 < cluster.localPos[0] < 10 and -5 < cluster.localPos[1] < 5) for cluster in event.clusters])): #cuts out multiple scattering with plastic window
                myEvents.append(event)

for event in myEvents:
    #200MeV 100e
    #if 85 < event.vertex.openingAngle*360/(2*math.pi) < 90 and abs(event.vertex.dca2origin - 18.75) < 2 and event.vertex.dca < 2 and all([5 < cluster.size < 25 for cluster in event.clusters]):#2sigma of 80MeV 100e target sizes
    #    if all([(-8 < cluster.localPos[0] < 8 and -3 < cluster.localPos[1] < 3) for cluster in event.clusters]):
    #
    #80MeV 100e
    #if 87 < event.vertex.openingAngle*360/(2*math.pi) < 92 and abs(event.vertex.dca2origin - 18.75) < 2 and event.vertex.dca < 2 and all([10 < cluster.size < 30 for cluster in event.clusters]):#2sigma of 80MeV 100e target sizes

    TrackLeft = Track()
    TrackLeft.fromClusters([event.selectDetector(alpide)[0].alignLocal(localxupdated_alignment.get(alpide)) for alpide in ["ALPIDE_3","ALPIDE_4"]])
    TrackRight = Track()
    TrackRight.fromClusters([event.selectDetector(alpide)[0].alignLocal(localxupdated_alignment.get(alpide)) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])

    #TrackLeft = Track()
    #TrackLeft.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_3","ALPIDE_4"]])
    #TrackRight = Track()
    #TrackRight.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
    
    event.vertex = Vertex()
    event.vertex.fromTracks([TrackLeft,TrackRight])
    
    verySelectedEvents.append(event)
    
    if (index%1000 ==0): print(str(index/len(tempEvents)*100)[:4], "%", end="\r")
    dcaVals.append(event.vertex.dca)
    dcaValsX.append(event.vertex.dcaVec[0])
    dcaValsY.append(event.vertex.dcaVec[1])
    dcaValsZ.append(event.vertex.dcaVec[2])
    for track in event.tracks:
        if track.nClusters == 3:
            trackRMS = track.rms
            trackRMSVals.append(trackRMS)

    
    hVertexXY.Fill(event.vertex.point[0],event.vertex.point[1]) #maybe apply some selection to the tracks, ask berkin
    hVertexXZ.Fill(event.vertex.point[0],event.vertex.point[2])
    hVertexYZ.Fill(event.vertex.point[1],event.vertex.point[2])
    
print("Number of very selected events: ", len(verySelectedEvents))



canvas = ROOT.TCanvas("","",1300,500)
canvas.Divide(3)

canvas.cd(1)
ROOT.gPad.SetGrid(2)
hVertexXY.Draw("COLZ") 

canvas.cd(2)
ROOT.gPad.SetGrid(2)
hVertexXZ.Draw("COLZ")

canvas.cd(3)
ROOT.gPad.SetGrid(2)
hVertexYZ.Draw("COLZ")

canvas.Draw()
evaluateEventSet(myEvents)
#canvas.SaveAs("selected_events_canvas.png")
#canvas.Close()

Number of very selected events:  8160
Evaluated displacement of  {'ALPIDE_0': 0.34375, 'ALPIDE_1': 0.40625, 'ALPIDE_2': 0.375, 'ALPIDE_3': 0.0, 'ALPIDE_4': 0.251953125} {'ALPIDE_0': [0.5, -0.0546875, 0], 'ALPIDE_1': [0.65, -0.0546875, 0], 'ALPIDE_2': [0.75, -0.06875, 0], 'ALPIDE_3': [0.2, -0.125, 0], 'ALPIDE_4': [0.5546875, -0.05, 0]} with  8160  events. 
 Median DCA:  0.063146 
 Mean DCA:  0.141540 
 DCA sigma:  0.454342 
 Median track RMS:  0.482404
plotted


Warning in <TROOT::Append>: Replacing existing TH1: VertexXY (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: VertexXZ (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: VertexYZ (Potential memory leak).


In [120]:
evaluateEventSet(myEvents)

Evaluated displacement of  {'ALPIDE_0': 0.34375, 'ALPIDE_1': 0.40625, 'ALPIDE_2': 0.375, 'ALPIDE_3': 0.0, 'ALPIDE_4': 0.251953125} {'ALPIDE_0': [0.5, -0.0546875, 0], 'ALPIDE_1': [0.65, -0.0546875, 0], 'ALPIDE_2': [0.75, -0.06875, 0], 'ALPIDE_3': [0.2, -0.125, 0], 'ALPIDE_4': [0.5546875, -0.05, 0]} with  8160  events. 
 Median DCA:  0.063146 
 Mean DCA:  0.141540 
 DCA sigma:  0.454342 
 Median track RMS:  0.482404
plotted


## Openning angle

In [121]:
%jsroot on
hOpeningAngle = ROOT.TH1F("Angle","Angle",300,60,120)
for event in verySelectedEvents: #tempEvents: #
    hOpeningAngle.Fill(event.vertex.openingAngle*360/(2*math.pi))
canvas = ROOT.TCanvas()
hOpeningAngle.Draw("COLZ")
canvas.Draw()

Warning in <TROOT::Append>: Replacing existing TH1: Angle (Potential memory leak).


## DCA Values

In [122]:
%jsroot on
hDCAstdDev = ROOT.TH1F("DCA","DCA",2501,0,3)
hDCAstdDevX = ROOT.TH1F("DCA X","DCA X",2501,-1,1)
hDCAstdDevY = ROOT.TH1F("DCA Y","DCA Y",2501,-1,1)
hDCAstdDevZ = ROOT.TH1F("DCA Z","DCA Z",2501,-1,1)

for index, dca in enumerate(dcaVals): #verySelectedEvents:
    hDCAstdDev.Fill(dca)
    hDCAstdDevX.Fill(dcaValsX[index])
    hDCAstdDevY.Fill(dcaValsY[index])
    hDCAstdDevZ.Fill(dcaValsZ[index])

canvas = ROOT.TCanvas("","",800,600)
canvas.Divide(2,2)

canvas.cd(1)
ROOT.gPad.SetGrid(2)
hDCAstdDev.Draw("COLZ") 

canvas.cd(2)
ROOT.gPad.SetGrid(2)
hDCAstdDevX.Draw("COLZ")

canvas.cd(3)
ROOT.gPad.SetGrid(2)
hDCAstdDevY.Draw("COLZ")

canvas.cd(4)
ROOT.gPad.SetGrid(2)
hDCAstdDevZ.Draw("COLZ")

canvas.Draw()

Warning in <TROOT::Append>: Replacing existing TH1: DCA (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: DCA X (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: DCA Y (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: DCA Z (Potential memory leak).


In [ ]:
%jsroot on
hDCAstdDev = ROOT.TH1F("DCA","DCA",251,0,max(dcaVals)+1)

DCAnoTail = [dca <= np.median(dcaVals)+ np.std(dcaVals) for dca in dcaVals]
for dca in DCAnoTail:
    hDCAstdDev.Fill(dca)

canvas = ROOT.TCanvas("","",800,600)

#canvas.cd()
ROOT.gPad.SetGrid(2)
hDCAstdDev.Draw("COLZ")
canvas.Draw()

In [ ]:
nBins = {"x-Axis" : 600, "y-Axis" : 300, "z-Axis" : 300}
nRange = {"x-Axis" : 15, "y-Axis" : 7.5, "z-Axis" : 7.5}


hLine = ROOT.TH3F("th3","5 Point Tracks",150, -40, 40, 150, -40, 40, 150, -25, 30)
myGraphs = []
targetDistance = 18.75
allTracks=[]
for i, event in enumerate(tempEvents):
    if i > 1500: break
    myClusters = []
    for cluster in event.clusters: #TrackLeft, TrackRight
        #if(i%100==0): print(track.point)
        myCluster = copy.copy(cluster)
        #if myCluster.detector in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]:
        #    myCluster.globalPos = [cluster.globalPos[0], cluster.globalPos[1], cluster.globalPos[2]*-1 -targetDistance]
        #else:
        myCluster.globalPos = [cluster.globalPos[0], cluster.globalPos[1], cluster.globalPos[2]]#+targetDistance]
        myClusters.append(myCluster)
    track = Track()
    track.fromClusters(myClusters)
    allTracks.append(track)
    graph = ROOT.TGraph2D()

    for i,cluster in enumerate(myClusters):
        projectedPoint = track.line.project_point(cluster.globalPos)
        graph.SetPoint(i,projectedPoint[0], projectedPoint[1], projectedPoint[2])
        hLine.Fill(cluster.globalPos[0], cluster.globalPos[1], cluster.globalPos[2])
    hLine.Fill(event.vertex.point[0], event.vertex.point[1], event.vertex.point[2])
    myGraphs.append(graph)
canvas = ROOT.TCanvas()
hLine.Draw("BOX3") #"LINE")
#for graph in myGraphs:
 #   graph.Draw("SAME LINE")

canvas.Draw()

In [ ]:
#nBins = {"x-Axis" : 600, "y-Axis" : 300, "z-Axis" : 300}
#nRange = {"x-Axis" : 15, "y-Axis" : 7.5, "z-Axis" : 7.5}
#hLine = ROOT.TH3F("th3","5 Point Tracks",150, -40, 40, 150, -40, 40, 150, -40, 40)

nBins = {"x-Axis" : 600, "y-Axis" : 300, "z-Axis" : 300}
nRange = {"x-Axis" : 15, "y-Axis" : 7.5, "z-Axis" : 7.5}
hLine = ROOT.TH3F("th3","5 Point Tracks",150, -30, 30, 150, -30, 30, 150, -30, 30)

myGraphs = []
targetDistance = 18.75
allTracks=[]
for i, event in enumerate(verySelectedEvents):
    if i > 500: break
    myClusters = []
    for cluster in event.clusters: #TrackLeft, TrackRight
        #if(i%100==0): print(track.point)
        myCluster = copy.copy(cluster)
        if myCluster.detector in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]:
            myCluster.globalPos = [cluster.globalPos[0], cluster.globalPos[1], cluster.globalPos[2]]#*-1 -targetDistance]
        else:
            myCluster.globalPos = [cluster.globalPos[0], cluster.globalPos[1], cluster.globalPos[2]]#+targetDistance]
        myClusters.append(myCluster)
    track = Track()
    track.fromClusters(myClusters)
    allTracks.append(track)
    graph = ROOT.TGraph2D()
    for i,cluster in enumerate(myClusters):
        projectedPoint = track.line.project_point(cluster.globalPos)
        graph.SetPoint(i,projectedPoint[0], projectedPoint[1], projectedPoint[2])
        hLine.Fill(cluster.globalPos[0], cluster.globalPos[1], cluster.globalPos[2])
    hLine.Fill(event.vertex.point[0], event.vertex.point[1], event.vertex.point[2])

    myGraphs.append(graph)
canvas = ROOT.TCanvas()
hLine.Draw("BOX3")
#for graph in myGraphs:
#    graph.Draw("SAME LINE")

canvas.Draw()

In [13]:
%jsroot on
    
def evaluateEventSet(eventSet):
    hVertexXY = ROOT.TH2F("VertexXY","VertexXY",200,-7,7,200,-7,7)
    hVertexXY.GetXaxis().SetTitle("Vertex X position [mm]")
    hVertexXY.GetYaxis().SetTitle("Vertex Y position [mm]")
    hVertexXZ = ROOT.TH2F("VertexXZ","VertexXZ",120,-2,2,120,-21,-17)
    hVertexXZ.GetXaxis().SetTitle("Vertex X position [mm]")
    hVertexXZ.GetYaxis().SetTitle("Vertex Z position [mm]")
    hVertexYZ = ROOT.TH2F("VertexYZ","VertexYZ",200,-7,7,200,-26,-12)
    hVertexYZ.GetXaxis().SetTitle("Vertex Y position [mm]")
    hVertexYZ.GetYaxis().SetTitle("Vertex Z position [mm]")
    mydcaVals=[]
    for index, event in enumerate(eventSet):  
        
        #TrackLeft = Track()
        #TrackLeft.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_3","ALPIDE_4"]])
        #TrackRight = Track()
        #TrackRight.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])

        #vertex = Vertex()
        #vertex.fromTracks([TrackLeft,TrackRight])
        if (index%1000 ==0): print(str(index/len(tempEvents)*100)[:4], "%", end="\r")
        #event.addTrack(TrackLeft)
        #event.addTrack(TrackRight)
        #event.vertex = vertex
        #tempEvents.append(event)
        mydcaVals.append(event.vertex.dca)
        for track in event.tracks:
            if track.nClusters == 3:
                trackRMS = track.rms
                trackRMSVals.append(trackRMS)

        hVertexXY.Fill(event.vertex.point[0],event.vertex.point[1]) #maybe apply some selection to the tracks, ask berkin
        hVertexXZ.Fill(event.vertex.point[0],event.vertex.point[2])
        hVertexYZ.Fill(event.vertex.point[1],event.vertex.point[2])
    medianDCA = np.median(mydcaVals)
    meanDCA = np.mean(mydcaVals)
    medianTrackRMS = np.median(trackRMSVals)
    print("Evaluated displacement of ", displacement, best_displacement_no_multiple_scat, "with ", len(eventSet)," events. \n Median DCA: ",str(medianDCA)[:8],"\n Mean DCA: ", str(meanDCA)[:8], "\n DCA sigma: ", str(np.std(dcaVals))[:8], "\n Median track RMS: ",str(medianTrackRMS)[:8])

    canvas = ROOT.TCanvas("","",1300,500)
    canvas.Divide(3)

    canvas.cd(1)
    ROOT.gPad.SetGrid(2)
    hVertexXY.Draw("COLZ") 

    canvas.cd(2)
    ROOT.gPad.SetGrid(2)
    hVertexXZ.Draw("COLZ")

    canvas.cd(3)
    ROOT.gPad.SetGrid(2)
    hVertexYZ.Draw("COLZ")

    canvas.Draw()
    print("plotted")
    
    